# Pre-Equilibrium Maximazation of Yield #

In [1]:
# make sure jupyter path is correct for loading local moudules
import sys
# path to steric_simulator module relative to notebook
sys.path.append("../../")
import copy

In [2]:
from steric_free_simulator import ReactionNetwork, VectorizedRxnNet, VecSim, Optimizer, EquilibriumSolver

EnergyExplorer Module is not available. Check Rosetta installation. <ipykernel.iostream.OutStream object at 0x7f4d8b32e978>


We'll start with the AP2 complex that we've worked with before. Pairwise $\Delta Gs$ were derived from the PDB structures via Rossetta

In [3]:
base_input = '../input_files/trimer.pwr'
rn = ReactionNetwork(base_input, one_step=True)
print(rn.rxn_coupling)
rn.resolve_tree()
print(rn.rxn_coupling)

['default_assoc', 1.0]
['A']
['B']
['C']
Parsing rule...
['A(a)+B(a)<->A(a!1).B(a!1)', 'G=-20']
['A', 'B', '']
['A(a)+B(a)', 'A(a!1).B(a!1)']
GGGGGGGGGgg
Parsing rule...
['A(b)+C(b)<->A(b!1).C(b!1)', 'G=-20']
['A', 'C', '']
['A(b)+C(b)', 'A(b!1).C(b!1)']
GGGGGGGGGgg
Parsing rule...
['B(b)+C(a)<->B(b!1).C(a!1)', 'G=-20']
['B', 'C', '']
['B(b)+C(a)', 'B(b!1).C(a!1)']
GGGGGGGGGgg
False
Node-1 :  (0, {'struct': <networkx.classes.graph.Graph object at 0x7f4cf4883978>, 'copies': tensor([100.], dtype=torch.float64)})
Node-2 :  (0, {'struct': <networkx.classes.graph.Graph object at 0x7f4cf4883978>, 'copies': tensor([100.], dtype=torch.float64)})
-----
{'A'}
{'A'}
set()
Steric hindrance detected
Node-1 :  (0, {'struct': <networkx.classes.graph.Graph object at 0x7f4cf4883978>, 'copies': tensor([100.], dtype=torch.float64)})
Node-2 :  (1, {'struct': <networkx.classes.graph.Graph object at 0x7f4cf549abe0>, 'copies': tensor([100.], dtype=torch.float64)})
-----
{'A'}
{'B'}
{'A'}
False
Allowed edges:

In [4]:
def decompose_monomers(n1,monomer_set):
        if len(rn.network.in_edges(n1)) == 0:
            return(True,monomer_set)
        else:
            for incoming_edge in rn.network.in_edges(n1):
                flag,monomer_set = decompose_monomers(incoming_edge[0],monomer_set)
                if flag:
                    monomer_set.append(incoming_edge[0])
            return(False,monomer_set)

In [5]:
uid_dict = {}
sys.path.append("../")
import numpy as np
from reaction_network import gtostr
for n in rn.network.nodes():
    print(n)
    #print(rn.network.nodes()[n])
    
    flag,monomer_set = decompose_monomers(n,[])
    
    print("Node - ", gtostr(rn.network.nodes[n]['struct']), "--", set(monomer_set))
    for k,v in rn.network[n].items():
        uid = v['uid']
        r1 = set(gtostr(rn.network.nodes[n]['struct']))
        p = set(gtostr(rn.network.nodes[k]['struct']))
        r2 = p-r1
        reactants = (r1,r2)
        uid_dict[(n,k)] = uid

print(uid_dict)
# print(rn.mon_rxns)
# print(gtostr(rn.network.nodes[3]['struct']),rn.network.nodes[3]['struct'].number_of_edges())

# print(rn.network.edges(data=True))

# print(rn.uid_map)

print(rn.rxn_cid)

0
Node -  A -- set()
1
Node -  B -- set()
2
Node -  C -- set()
3
Node -  AB -- {0, 1}
4
Node -  AC -- {0, 2}
5
Node -  BC -- {1, 2}
6
Node -  ABC -- {0, 1, 2}
{(0, 3): 0, (0, 4): 1, (0, 6): 5, (1, 3): 0, (1, 5): 2, (1, 6): 3, (2, 4): 1, (2, 5): 2, (2, 6): 4, (3, 6): 4, (4, 6): 3, (5, 6): 5}
{}


In [6]:
#Changing Initial Conditions
import networkx as nx
#Changin k_on
new_kon = [1.0,1.0,1.0,1.0,1.0,1.0]*np.array([1e0])  #Poor yield Simruntime = 1sec
# new_kon = [ 0.1411,  0.1917,  0.1205, 42.6304, 42.7995, 42.6389]*np.array([1e0])
# new_kon = [0.4052, 0.4145, 0.4049, 5.0401, 5.9947, 5.9940]

update_kon_dict = {}
for edge in rn.network.edges:
    print(rn.network.get_edge_data(edge[0],edge[1]))
    update_kon_dict[edge] = new_kon[uid_dict[edge]]

nx.set_edge_attributes(rn.network,update_kon_dict,'k_on')

# for edge in rn.network.edges:
#     print(rn.network.get_edge_data(edge[0],edge[1]))
# vec_rn = VectorizedRxnNet(rn, dev='cpu',rxn_coupling=True,rx_cid={3:[1],4:[0],5:[2]})
vec_rn = VectorizedRxnNet(rn, dev='cpu')
print(vec_rn.kon)


{'k_on': 1.0, 'k_off': None, 'lcf': 1, 'rxn_score': tensor([-20.], dtype=torch.float64), 'uid': 0}
{'k_on': 1.0, 'k_off': None, 'lcf': 1, 'rxn_score': tensor([-20.], dtype=torch.float64), 'uid': 1}
{'k_on': 1.0, 'k_off': None, 'lcf': 1, 'rxn_score': tensor([-40.], dtype=torch.float64), 'uid': 5}
{'k_on': 1.0, 'k_off': None, 'lcf': 1, 'rxn_score': tensor([-20.], dtype=torch.float64), 'uid': 0}
{'k_on': 1.0, 'k_off': None, 'lcf': 1, 'rxn_score': tensor([-20.], dtype=torch.float64), 'uid': 2}
{'k_on': 1.0, 'k_off': None, 'lcf': 1, 'rxn_score': tensor([-40.], dtype=torch.float64), 'uid': 3}
{'k_on': 1.0, 'k_off': None, 'lcf': 1, 'rxn_score': tensor([-20.], dtype=torch.float64), 'uid': 1}
{'k_on': 1.0, 'k_off': None, 'lcf': 1, 'rxn_score': tensor([-20.], dtype=torch.float64), 'uid': 2}
{'k_on': 1.0, 'k_off': None, 'lcf': 1, 'rxn_score': tensor([-40.], dtype=torch.float64), 'uid': 4}
{'k_on': 1.0, 'k_off': None, 'lcf': 1, 'rxn_score': tensor([-40.], dtype=torch.float64), 'uid': 4}
{'k_on': 1

## The Equilibrium Solution ##
First we will find the equilibrium solution for this system.

In [7]:
vec_rn.update_reaction_net(rn,1)
poly_system = EquilibriumSolver(rn)
solution = poly_system.solve()
if solution == None:
    print("No Equilibrium solution")
else:
    print(solution)
    print("Equilibrium expected yield: ", 100 * solution[-1] / min(vec_rn.initial_copies[:vec_rn.num_monomers]), '%')
print(vec_rn.kon)

No aceeptable solution found
No Equilibrium solution
Parameter containing:
tensor([1., 1., 1., 1., 1., 1.], dtype=torch.float64, requires_grad=True)


In [8]:
print(vec_rn.rxn_coupling)
print(rn.rxn_coupling)

False
False


In [9]:
uid_dict = {}
sys.path.append("../")
import numpy as np
from reaction_network import gtostr
for n in rn.network.nodes():
    #print(n)
    #print(rn.network.nodes()[n])
    for k,v in rn.network[n].items():
        uid = v['uid']
        r1 = set(gtostr(rn.network.nodes[n]['struct']))
        p = set(gtostr(rn.network.nodes[k]['struct']))
        r2 = p-r1
        reactants = (r1,r2)
        uid_val = {'reactants':reactants,'kon':v['k_on'],'score':v['rxn_score'],'koff':v['k_off'],'uid':uid}
        if uid not in uid_dict.keys():
            uid_dict[uid] = uid_val
    print(gtostr(rn.network.nodes[n]['struct']))
    #for r_set in rn.get_reactant_sets(n):
    #    print(tuple(r_set))
    #print(rn.network[n]['struct'])
ind_sort = np.argsort(vec_rn.kon.detach().numpy())
for i in ind_sort:
    print(vec_rn.kon[i])
    print(uid_dict[i])

A
B
C
AB
AC
BC
ABC
tensor(1., dtype=torch.float64, grad_fn=<SelectBackward>)
{'reactants': ({'A'}, {'B'}), 'kon': 1.0, 'score': tensor([-20.], dtype=torch.float64), 'koff': 0.002061153622438557, 'uid': 0}
tensor(1., dtype=torch.float64, grad_fn=<SelectBackward>)
{'reactants': ({'A'}, {'C'}), 'kon': 1.0, 'score': tensor([-20.], dtype=torch.float64), 'koff': 0.002061153622438557, 'uid': 1}
tensor(1., dtype=torch.float64, grad_fn=<SelectBackward>)
{'reactants': ({'B'}, {'C'}), 'kon': 1.0, 'score': tensor([-20.], dtype=torch.float64), 'koff': 0.002061153622438557, 'uid': 2}
tensor(1., dtype=torch.float64, grad_fn=<SelectBackward>)
{'reactants': ({'B'}, {'A', 'C'}), 'kon': 1.0, 'score': tensor([-40.], dtype=torch.float64), 'koff': 4.248354255291587e-12, 'uid': 3}
tensor(1., dtype=torch.float64, grad_fn=<SelectBackward>)
{'reactants': ({'C'}, {'A', 'B'}), 'kon': 1.0, 'score': tensor([-40.], dtype=torch.float64), 'koff': 4.248354255291587e-12, 'uid': 4}
tensor(1., dtype=torch.float64, grad_fn

In [10]:
uid_dict = {}
sys.path.append("../")
import numpy as np
from reaction_network import gtostr
from torch import DoubleTensor as Tensor
import torch

node_map = {}
for node in rn.network.nodes():
    node_map[gtostr(rn.network.nodes[node]['struct'])] = node

print(node_map)

def get_max_edge(n):
    """
    Calculates the max rate (k_on) for a given node
    To find out the maximum flow path to the final complex starting from the current node.
    
    Can also calculate the total rate of consumption of a node by summing up all rates. 
    Can tell which component is used quickly.
    """
    try:
        edges = rn.network.out_edges(n)
        #Loop over all edges
        #Get attributes
        if len(edges)==0:
            return(False)
        kon_max = -1
        next_node = -1
        
        kon_sum = 0
        for edge in edges:
            data = rn.network.get_edge_data(edge[0],edge[1])
            #print(data)
            #Get uid
            uid = data['uid']
            #Get updated kon
            temp_kon = vec_rn.kon[uid]
            kon_sum+=temp_kon
            
            if temp_kon > kon_max:
                kon_max = temp_kon
                next_node=edge[1]
        return(kon_max,next_node,kon_sum)
    except Exception as err:
        raise(err)
def get_node_flux(n):
    total_flux_outedges = 0
    total_flux_inedges = 0
    #Go over all the out edges
    edges_out = rn.network.out_edges(n)
    if len(edges_out)>0:

        for edge in edges_out:
            data = rn.network.get_edge_data(edge[0],edge[1])
            #print(data)
            #Get uid
            uid = data['uid']

            #Get updated kon
            temp_kon = vec_rn.kon[uid]

            #Calculate k_off also
            std_c = Tensor([1.])
            l_kon = torch.log(temp_kon)
            l_koff = (vec_rn.rxn_score_vec[uid] * 1. / (vec_rn._R * vec_rn._T)) + l_kon + torch.log(std_c)
            koff = torch.exp(l_koff)

            #Getting conc. of reactants and products
            #Get product
            prod = gtostr(rn.network.nodes[edge[1]]['struct']) 
            #Get other reactant
            react = "".join(sorted(list(set(prod) - set(gtostr(rn.network.nodes[edge[0]]['struct']) ))))

            #Net flux from this edge = Generation - consumption
            edge_flux = koff*solution[edge[1]] - temp_kon*(solution[edge[0]])*(solution[node_map[react]])
            #edge_flux = koff*vec_rn.copies_vec[edge[1]] 

            print("Reaction: ", gtostr(rn.network.nodes[edge[0]]['struct']), "+",react," -> ",prod)
            print("Net flux: ",edge_flux)
            print("kon : ",temp_kon)
            print("koff: ",koff)
            print("Free Energy: ",vec_rn.rxn_score_vec[uid])
            print("Reaction data OUTWARD: ")
            print(data)

            total_flux_outedges+=edge_flux
    
    #Now go over all the in edges
    edges_in = rn.network.in_edges(n)
    react_list = []
    if len(edges_in) > 0:
        for edge in edges_in:
            if edge[0] in react_list:
                continue
            data = rn.network.get_edge_data(edge[0],edge[1])
            uid = data['uid']


            #Get generation rates; which would be kon
            temp_kon = vec_rn.kon[uid]

            #Get consumption rates; which is k_off
            std_c = Tensor([1.])
            l_kon = torch.log(temp_kon)
            l_koff = (vec_rn.rxn_score_vec[uid] * 1. / (vec_rn._R * vec_rn._T)) + l_kon + torch.log(std_c)
            koff = torch.exp(l_koff)

            #Get conc. of reactants and products
            prod = gtostr(rn.network.nodes[edge[1]]['struct'])
            #Get other reactant
            react = "".join(sorted(list(set(prod) - set(gtostr(rn.network.nodes[edge[0]]['struct']) ))))
            react_list.append(node_map[react])
            #Net flux from this edge = Generation - consumption
            edge_flux_in = temp_kon*(solution[edge[0]])*(solution[node_map[react]])- koff*solution[edge[1]]
            #edge_flux_in = koff*vec_rn.copies_vec[edge[1]]
            


            print("Reaction: ", prod ," -> ",gtostr(rn.network.nodes[edge[0]]['struct']), "+",react)
            print("Net flux: ",edge_flux_in)
            print("kon : ",temp_kon)
            print("koff: ",koff)
            print("Raction data INWARD: ")
            print(data)

            total_flux_inedges+=edge_flux_in
    net_node_flux = total_flux_outedges + total_flux_inedges
    
    return(net_node_flux)
pathway = []
kon_sumarray = []
total_con_rate = {}
net_flux = {}

if solution != None:
    for n in rn.network.nodes():

        n_str = gtostr(rn.network.nodes[n]['struct']) 

        paths = [n_str]
        kon_sum = 0
        temp_node = n
        max_edge = True
        consumption_rate = 0
        if n < len(rn.network.nodes()):#num_monomers:
    #         print("Current node: ")
    #         print(n_str)
            while max_edge:
                max_edge = get_max_edge(temp_node)
                if max_edge:
                    total_con_rate[gtostr(rn.network.nodes[temp_node]['struct'])] = max_edge[2]
                    temp_node = max_edge[1]
                    kon_sum += max_edge[0].item()


    #                 print("Next node: ")
    #                 print(temp_node)

                    paths.append(gtostr(rn.network.nodes[temp_node]['struct']))
                else:
                    break
            pathway.append(paths)
            kon_sumarray.append(kon_sum)
            paths=[]

        print("-------------------------------------------------------------------------------")
        print("-------------------------------------------------------------------------------")
        print("|                                                                             |")
        node_flux = get_node_flux(n)
        net_flux[gtostr(rn.network.nodes[n]['struct'])] = node_flux
        print("|                                                                             |")
        print("-------------------------------------------------------------------------------")
        print("-------------------------------------------------------------------------------")

    print(pathway)
    print(kon_sumarray)
    #print(total_con_rate)

{'A': 0, 'B': 1, 'C': 2, 'AB': 3, 'AC': 4, 'BC': 5, 'ABC': 6}


In [11]:
if solution != None:
    for k,v in sorted(total_con_rate.items(),key=lambda x : x[1]):
        print(k," : ", v.item())

In [12]:
if solution !=None:
    for k,v in sorted(net_flux.items(),key=lambda x : x[1]):
        print(k," : ", v)



## Using the optimizer with a 1 second simulation runtime ##

In [13]:
vec_rn.reset(reset_params=True)
optim = Optimizer(reaction_network=vec_rn,
                  sim_runtime=1,
                  optim_iterations=2000,
                  learning_rate=new_kon[0]/1e2,
                  device='cpu',method="RMSprop")
# optim.rn.update_reaction_net(rn)
optim.optimize(optim='flux_coeff',corr_rxns=[[4],[9]])

Using CPU
Reaction Parameters before optimization: 
[Parameter containing:
tensor([1., 1., 1., 1., 1., 1.], dtype=torch.float64, requires_grad=True)]
Using CPU
[[4], [9]]
Optimizing Flux Correlations
yield on sim iteration 0 was -0.6254485823962923
current params: tensor([1.1000, 1.1000, 1.1000, 1.0000, 1.0000, 1.0000], dtype=torch.float64)
Using CPU
[[4], [9]]
Optimizing Flux Correlations
yield on sim iteration 1 was -0.6210743413381352
current params: tensor([1.1709, 1.1709, 1.1709, 1.0001, 1.0001, 1.0001], dtype=torch.float64)
Using CPU
[[4], [9]]
Optimizing Flux Correlations
yield on sim iteration 2 was -0.6152492621844069
current params: tensor([1.2289, 1.2289, 1.2289, 1.0001, 1.0001, 1.0001], dtype=torch.float64)
Using CPU
[[4], [9]]
Optimizing Flux Correlations
yield on sim iteration 3 was -0.6085834084366055
current params: tensor([1.2793, 1.2793, 1.2793, 1.0002, 1.0002, 1.0002], dtype=torch.float64)
Using CPU
[[4], [9]]
Optimizing Flux Correlations
yield on sim iteration 4 was

[[4], [9]]
Optimizing Flux Correlations
yield on sim iteration 42 was -0.5778061919848297
current params: tensor([2.2185, 2.2185, 2.2185, 1.0018, 1.0018, 1.0018], dtype=torch.float64)
Using CPU
[[4], [9]]
Optimizing Flux Correlations
yield on sim iteration 43 was -0.5784786977610059
current params: tensor([2.2352, 2.2352, 2.2352, 1.0019, 1.0019, 1.0019], dtype=torch.float64)
Using CPU
[[4], [9]]
Optimizing Flux Correlations
yield on sim iteration 44 was -0.5791412182055381
current params: tensor([2.2518, 2.2518, 2.2518, 1.0019, 1.0019, 1.0019], dtype=torch.float64)
Using CPU
[[4], [9]]
Optimizing Flux Correlations
yield on sim iteration 45 was -0.5797940876315788
current params: tensor([2.2682, 2.2682, 2.2682, 1.0020, 1.0020, 1.0020], dtype=torch.float64)
Using CPU
[[4], [9]]
Optimizing Flux Correlations
yield on sim iteration 46 was -0.5804376118214177
current params: tensor([2.2845, 2.2845, 2.2845, 1.0020, 1.0020, 1.0020], dtype=torch.float64)
Using CPU
[[4], [9]]
Optimizing Flux Cor

[[4], [9]]
Optimizing Flux Correlations
yield on sim iteration 85 was -0.5641730272098593
current params: tensor([2.8466, 2.8466, 2.8466, 1.0036, 1.0036, 1.0036], dtype=torch.float64)
Using CPU
[[4], [9]]
Optimizing Flux Correlations
yield on sim iteration 86 was -0.564611488282435
current params: tensor([2.8597, 2.8597, 2.8597, 1.0037, 1.0037, 1.0037], dtype=torch.float64)
Using CPU
[[4], [9]]
Optimizing Flux Correlations
yield on sim iteration 87 was -0.5650464324358899
current params: tensor([2.8728, 2.8728, 2.8728, 1.0037, 1.0037, 1.0037], dtype=torch.float64)
Using CPU
[[4], [9]]
Optimizing Flux Correlations
yield on sim iteration 88 was -0.5654779335873725
current params: tensor([2.8858, 2.8858, 2.8858, 1.0038, 1.0038, 1.0038], dtype=torch.float64)
Using CPU
[[4], [9]]
Optimizing Flux Correlations
yield on sim iteration 89 was -0.5659060455178623
current params: tensor([2.8987, 2.8987, 2.8987, 1.0038, 1.0038, 1.0038], dtype=torch.float64)
Using CPU
[[4], [9]]
Optimizing Flux Corr

[[4], [9]]
Optimizing Flux Correlations
yield on sim iteration 128 was -0.5676551392377415
current params: tensor([3.3769, 3.3769, 3.3769, 1.0055, 1.0055, 1.0055], dtype=torch.float64)
Using CPU
[[4], [9]]
Optimizing Flux Correlations
yield on sim iteration 129 was -0.5679882781353818
current params: tensor([3.3886, 3.3886, 3.3886, 1.0055, 1.0055, 1.0055], dtype=torch.float64)
Using CPU
[[4], [9]]
Optimizing Flux Correlations
yield on sim iteration 130 was -0.5683195184584295
current params: tensor([3.4003, 3.4003, 3.4003, 1.0056, 1.0056, 1.0056], dtype=torch.float64)
Using CPU
[[4], [9]]
Optimizing Flux Correlations
yield on sim iteration 131 was -0.5553529323706846
current params: tensor([3.4119, 3.4119, 3.4119, 1.0056, 1.0056, 1.0056], dtype=torch.float64)
Using CPU
[[4], [9]]
Optimizing Flux Correlations
yield on sim iteration 132 was -0.5556857187166402
current params: tensor([3.4236, 3.4236, 3.4236, 1.0056, 1.0056, 1.0056], dtype=torch.float64)
Using CPU
[[4], [9]]
Optimizing Flu

[[4], [9]]
Optimizing Flux Correlations
yield on sim iteration 171 was -0.5537845872507012
current params: tensor([3.8643, 3.8643, 3.8643, 1.0073, 1.0073, 1.0073], dtype=torch.float64)
Using CPU
[[4], [9]]
Optimizing Flux Correlations
yield on sim iteration 172 was -0.5540626162095398
current params: tensor([3.8753, 3.8753, 3.8753, 1.0073, 1.0073, 1.0073], dtype=torch.float64)
Using CPU
[[4], [9]]
Optimizing Flux Correlations
yield on sim iteration 173 was -0.5543394344158109
current params: tensor([3.8863, 3.8863, 3.8863, 1.0074, 1.0074, 1.0074], dtype=torch.float64)
Using CPU
[[4], [9]]
Optimizing Flux Correlations
yield on sim iteration 174 was -0.5546150532970531
current params: tensor([3.8973, 3.8973, 3.8973, 1.0074, 1.0074, 1.0074], dtype=torch.float64)
Using CPU
[[4], [9]]
Optimizing Flux Correlations
yield on sim iteration 175 was -0.5548894854026684
current params: tensor([3.9083, 3.9083, 3.9083, 1.0075, 1.0075, 1.0075], dtype=torch.float64)
Using CPU
[[4], [9]]
Optimizing Flu

...

[[4], [9]]
Optimizing Flux Correlations
yield on sim iteration 1653 was -0.6449310039746198
current params: tensor([18.7786, 18.7786, 18.7786,  1.0700,  1.0700,  1.0700],
       dtype=torch.float64)
Using CPU
[[4], [9]]
Optimizing Flux Correlations
yield on sim iteration 1654 was -0.6449334171077518
current params: tensor([18.7886, 18.7886, 18.7886,  1.0700,  1.0700,  1.0700],
       dtype=torch.float64)
Using CPU
[[4], [9]]
Optimizing Flux Correlations
yield on sim iteration 1655 was -0.6449354118260148
current params: tensor([18.7986, 18.7986, 18.7986,  1.0701,  1.0701,  1.0701],
       dtype=torch.float64)
Using CPU
[[4], [9]]
Optimizing Flux Correlations
yield on sim iteration 1656 was -0.6449370091932831
current params: tensor([18.8086, 18.8086, 18.8086,  1.0701,  1.0701,  1.0701],
       dtype=torch.float64)
Using CPU
[[4], [9]]
Optimizing Flux Correlations
yield on sim iteration 1657 was -0.6449382237618781
current params: tensor([18.8186, 18.8186, 18.8186,  1.0702,  1.0702,  1.

[[4], [9]]
Optimizing Flux Correlations
yield on sim iteration 1693 was -0.6506478439421315
current params: tensor([19.1786, 19.1786, 19.1786,  1.0717,  1.0717,  1.0717],
       dtype=torch.float64)
Using CPU
[[4], [9]]
Optimizing Flux Correlations
yield on sim iteration 1694 was -0.6446910313316564
current params: tensor([19.1886, 19.1886, 19.1886,  1.0717,  1.0717,  1.0717],
       dtype=torch.float64)
Using CPU
[[4], [9]]
Optimizing Flux Correlations
yield on sim iteration 1695 was -0.6446202048952511
current params: tensor([19.1986, 19.1986, 19.1986,  1.0718,  1.0718,  1.0718],
       dtype=torch.float64)
Using CPU
[[4], [9]]
Optimizing Flux Correlations
yield on sim iteration 1696 was -0.6505313045814991
current params: tensor([19.2086, 19.2086, 19.2086,  1.0718,  1.0718,  1.0718],
       dtype=torch.float64)
Using CPU
[[4], [9]]
Optimizing Flux Correlations
yield on sim iteration 1697 was -0.638149863759408
current params: tensor([19.2186, 19.2186, 19.2186,  1.0718,  1.0718,  1.0

[[4], [9]]
Optimizing Flux Correlations
yield on sim iteration 1733 was -0.6447564122283217
current params: tensor([19.5786, 19.5786, 19.5786,  1.0734,  1.0734,  1.0734],
       dtype=torch.float64)
Using CPU
[[4], [9]]
Optimizing Flux Correlations
yield on sim iteration 1734 was -0.6447813001388815
current params: tensor([19.5886, 19.5886, 19.5886,  1.0734,  1.0734,  1.0734],
       dtype=torch.float64)
Using CPU
[[4], [9]]
Optimizing Flux Correlations
yield on sim iteration 1735 was -0.6448132959372948
current params: tensor([19.5986, 19.5986, 19.5986,  1.0735,  1.0735,  1.0735],
       dtype=torch.float64)
Using CPU
[[4], [9]]
Optimizing Flux Correlations
yield on sim iteration 1736 was -0.6383299039562601
current params: tensor([19.6086, 19.6086, 19.6086,  1.0735,  1.0735,  1.0735],
       dtype=torch.float64)
Using CPU
[[4], [9]]
Optimizing Flux Correlations
yield on sim iteration 1737 was -0.6506107949977303
current params: tensor([19.6186, 19.6186, 19.6186,  1.0735,  1.0735,  1.

[[4], [9]]
Optimizing Flux Correlations
yield on sim iteration 1773 was -0.6511849093039447
current params: tensor([19.9786, 19.9786, 19.9786,  1.0751,  1.0751,  1.0751],
       dtype=torch.float64)
Using CPU
[[4], [9]]
Optimizing Flux Correlations
yield on sim iteration 1774 was -0.6511811580066144
current params: tensor([19.9886, 19.9886, 19.9886,  1.0751,  1.0751,  1.0751],
       dtype=torch.float64)
Using CPU
[[4], [9]]
Optimizing Flux Correlations
yield on sim iteration 1775 was -0.6511771641917089
current params: tensor([19.9986, 19.9986, 19.9986,  1.0751,  1.0751,  1.0751],
       dtype=torch.float64)
Using CPU
[[4], [9]]
Optimizing Flux Correlations
yield on sim iteration 1776 was -0.6511729536946899
current params: tensor([20.0086, 20.0086, 20.0086,  1.0752,  1.0752,  1.0752],
       dtype=torch.float64)
Using CPU
[[4], [9]]
Optimizing Flux Correlations
yield on sim iteration 1777 was -0.6511685451413461
current params: tensor([20.0186, 20.0186, 20.0186,  1.0752,  1.0752,  1.

KeyboardInterrupt: 

In [ ]:
print(optim.yield_per_iter[-1])

In [ ]:
uid_dict = {}
sys.path.append("../")
import numpy as np
from reaction_network import gtostr
for n in rn.network.nodes():
    #print(n)
    #print(rn.network.nodes()[n])
    for k,v in rn.network[n].items():
        uid = v['uid']
        r1 = set(gtostr(rn.network.nodes[n]['struct']))
        p = set(gtostr(rn.network.nodes[k]['struct']))
        r2 = p-r1
        reactants = (r1,r2)
        uid_val = {'reactants':reactants,'kon':v['k_on'],'score':v['rxn_score'],'koff':v['k_off'],'uid':uid}
        if uid not in uid_dict.keys():
            uid_dict[uid] = uid_val
    print(gtostr(rn.network.nodes[n]['struct']))
    #for r_set in rn.get_reactant_sets(n):
    #    print(tuple(r_set))
    #print(rn.network[n]['struct'])
ind_sort = np.argsort(vec_rn.kon.detach().numpy())
for i in ind_sort:
    print(vec_rn.kon[i])
    print(uid_dict[i])

In [ ]:
uid_dict = {}
sys.path.append("../")
import numpy as np
from reaction_network import gtostr
from torch import DoubleTensor as Tensor
import torch

node_map = {}
for node in rn.network.nodes():
    node_map[gtostr(rn.network.nodes[node]['struct'])] = node

print(node_map)

def get_max_edge(n):
    """
    Calculates the max rate (k_on) for a given node
    To find out the maximum flow path to the final complex starting from the current node.
    
    Can also calculate the total rate of consumption of a node by summing up all rates. 
    Can tell which component is used quickly.
    """
    try:
        edges = rn.network.out_edges(n)
        #Loop over all edges
        #Get attributes
        kon_max = -1
        next_node = -1

        kon_sum = 0
        total_flux_outedges = 0
        total_flux_inedges = 0
        if len(edges)==0:
            return(False)
            
        for edge in edges:
            data = rn.network.get_edge_data(edge[0],edge[1])
            #print(data)
            #Get uid
            uid = data['uid']

            #Get updated kon
            temp_kon = vec_rn.kon[uid]
            kon_sum+=temp_kon
            
            if temp_kon > kon_max:
                kon_max = temp_kon
                next_node=edge[1]
             
        return(kon_max,next_node,kon_sum)
    except Exception as err:
        raise(err)

        
def get_node_flux(n):
    total_flux_outedges = 0
    total_flux_inedges = 0
    #Go over all the out edges
    edges_out = rn.network.out_edges(n)
    if len(edges_out)>0:

        for edge in edges_out:
            data = rn.network.get_edge_data(edge[0],edge[1])
            #print(data)
            #Get uid
            uid = data['uid']

            #Get updated kon
            temp_kon = vec_rn.kon[uid]

            #Calculate k_off also
            std_c = Tensor([1.])
            l_kon = torch.log(temp_kon)
            l_koff = (vec_rn.rxn_score_vec[uid] * 1. / (vec_rn._R * vec_rn._T)) + l_kon + torch.log(std_c)
            koff = torch.exp(l_koff)

            #Getting conc. of reactants and products
            #Get product
            prod = gtostr(rn.network.nodes[edge[1]]['struct']) 
            #Get other reactant
            react = "".join(sorted(list(set(prod) - set(gtostr(rn.network.nodes[edge[0]]['struct']) ))))

            #Net flux from this edge = Generation - consumption
            edge_flux = koff*vec_rn.copies_vec[edge[1]] - temp_kon*(vec_rn.copies_vec[edge[0]])*(vec_rn.copies_vec[node_map[react]])
            #edge_flux = koff*vec_rn.copies_vec[edge[1]] 

            print("Reaction: ", gtostr(rn.network.nodes[edge[0]]['struct']), "+",react," -> ",prod)
            print("Net flux: ",edge_flux)
            print("kon : ",temp_kon)
            print("koff: ",koff)
            print("Reaction data OUTWARD: ")
            print(data)

            total_flux_outedges+=edge_flux
    
    #Now go over all the in edges
    edges_in = rn.network.in_edges(n)
    react_list = []
    if len(edges_in) > 0:
        for edge in edges_in:
            if edge[0] in react_list:
                continue
            data = rn.network.get_edge_data(edge[0],edge[1])
            uid = data['uid']


            #Get generation rates; which would be kon
            temp_kon = vec_rn.kon[uid]

            #Get consumption rates; which is k_off
            std_c = Tensor([1.])
            l_kon = torch.log(temp_kon)
            l_koff = (vec_rn.rxn_score_vec[uid] * 1. / (vec_rn._R * vec_rn._T)) + l_kon + torch.log(std_c)
            koff = torch.exp(l_koff)

            #Get conc. of reactants and products
            prod = gtostr(rn.network.nodes[edge[1]]['struct'])
            #Get other reactant
            react = "".join(sorted(list(set(prod) - set(gtostr(rn.network.nodes[edge[0]]['struct']) ))))
            react_list.append(node_map[react])
            #Net flux from this edge = Generation - consumption
            edge_flux_in = temp_kon*(vec_rn.copies_vec[edge[0]])*(vec_rn.copies_vec[node_map[react]])- koff*vec_rn.copies_vec[edge[1]]
            #edge_flux_in = koff*vec_rn.copies_vec[edge[1]]
            


            print("Reaction: ", prod ," -> ",gtostr(rn.network.nodes[edge[0]]['struct']), "+",react)
            print("Net flux: ",edge_flux_in)
            print("kon : ",temp_kon)
            print("koff: ",koff)
            print("Raction data INWARD: ")
            print(data)

            total_flux_inedges+=edge_flux_in
    net_node_flux = total_flux_outedges + total_flux_inedges
    
    return(net_node_flux)
    
pathway = []
kon_sumarray = []
total_con_rate = {}
net_flux = {}
for n in rn.network.nodes():
    
    n_str = gtostr(rn.network.nodes[n]['struct']) 
    
    paths = [n_str]
    kon_sum = 0
    temp_node = n
    max_edge = True
    consumption_rate = 0
    if n < len(rn.network.nodes()):#num_monomers:
#         print("Current node: ")
#         print(n_str)
        while max_edge:
            max_edge = get_max_edge(temp_node)
            if max_edge:
                total_con_rate[gtostr(rn.network.nodes[temp_node]['struct'])] = max_edge[2]
                
                temp_node = max_edge[1]
                kon_sum += max_edge[0].item()
                
                
#                 print("Next node: ")
#                 print(temp_node)

                paths.append(gtostr(rn.network.nodes[temp_node]['struct']))
            else:
                break
        pathway.append(paths)
        kon_sumarray.append(kon_sum)
        paths=[]
    print("-------------------------------------------------------------------------------")
    print("-------------------------------------------------------------------------------")
    print("|                                                                             |")
    node_flux = get_node_flux(n)
    net_flux[gtostr(rn.network.nodes[n]['struct'])] = node_flux
    print("|                                                                             |")
    print("-------------------------------------------------------------------------------")
    print("-------------------------------------------------------------------------------")

print(pathway)
print(kon_sumarray)

#print(total_con_rate)

In [ ]:
for k,v in sorted(total_con_rate.items(),key=lambda x : x[1]):
    print(k," : ", v.item())

In [ ]:
for k,v in sorted(net_flux.items(),key=lambda x : x[1]):
    print(k," : ", v)

print(vec_rn.copies_vec)

In [ ]:
for n in rn.network.nodes():
    print(gtostr(rn.network.nodes[n]['struct']), " : ", n)

Let's first visualize some of the data.

**Without any optimization**


In [ ]:
nodes_list = ['A','B','C','ABC','AB','BC','AC']
#nodes_list = ['AB','BMS','ABS','AMS','ABMS','AM','BM','MS']
optim.plot_observable(0,nodes_list)


**After 750 optimization iterations**


In [ ]:

optim.plot_observable(-1,nodes_list)


In [ ]:
optim.plot_yield()

In [ ]:
import matplotlib.pyplot as plt
fig,ax = plt.subplots()
ax.plot(optim.yield_per_iter)
f_dict = {'fontsize':22}
ax.set_ylabel("Yield (%)",fontdict=f_dict)
ax.set_xlabel("Iterations",fontdict=f_dict)
plt.tick_params(axis='both',labelsize=14.0)
ax.hlines(y=0.999,xmin=0,xmax=5000,linewidth=2.5,linestyle='dashed',label='Eq. yield')
ax.legend(fontsize=16)

It seems like we've found a stable solution that produces greater yield than equilibrium. This should be thermodynamically
impossible. Let's try to find an explanation. We'll run simulations using the learned optimal parameters at a few different
timescales.

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline
fig, ax = plt.subplots(1, 4)
optim_rn = optim.rn
for i, runtime in enumerate([10, 100, 500,1000]):
    optim_rn.reset()
    sim = VecSim(optim_rn, runtime, device='cpu')
    y = sim.simulate()
    sim.plot_observable(nodes_list,ax=ax[i])
    ax[i].set_title("runtime: " + str(runtime) + " seconds")
fig.set_size_inches(18, 6)
plt.show()

In [ ]:
def get_max_edge(n):
    """
    Calculates the max rate (k_on) for a given node
    To find out the maximum flow path to the final complex starting from the current node.
    
    Can also calculate the total rate of consumption of a node by summing up all rates. 
    Can tell which component is used quickly.
    """
    try:
        edges = rn.network.out_edges(n)
        #Loop over all edges
        #Get attributes
        kon_max = -1
        next_node = -1

        kon_sum = 0
        total_flux_outedges = 0
        total_flux_inedges = 0
        if len(edges)==0:
            return(False)
            
        for edge in edges:
            data = rn.network.get_edge_data(edge[0],edge[1])
            #print(data)
            #Get uid
            uid = data['uid']

            #Get updated kon
            temp_kon = vec_rn.kon[uid]
            kon_sum+=temp_kon
            
            if temp_kon > kon_max:
                kon_max = temp_kon
                next_node=edge[1]
             
        return(kon_max,next_node,kon_sum)
    except Exception as err:
        raise(err)

        
def get_node_flux(n):
    total_flux_outedges = 0
    total_flux_inedges = 0
    #Go over all the out edges
    edges_out = rn.network.out_edges(n)
    if len(edges_out)>0:

        for edge in edges_out:
            data = rn.network.get_edge_data(edge[0],edge[1])
            #print(data)
            #Get uid
            uid = data['uid']

            #Get updated kon
            temp_kon = vec_rn.kon[uid]

            #Calculate k_off also
            std_c = Tensor([1.])
            l_kon = torch.log(temp_kon)
            l_koff = (vec_rn.rxn_score_vec[uid] * 1. / (vec_rn._R * vec_rn._T)) + l_kon + torch.log(std_c)
            koff = torch.exp(l_koff)

            #Getting conc. of reactants and products
            #Get product
            prod = gtostr(rn.network.nodes[edge[1]]['struct']) 
            #Get other reactant
            react = "".join(sorted(list(set(prod) - set(gtostr(rn.network.nodes[edge[0]]['struct']) ))))

            #Net flux from this edge = Generation - consumption
            edge_flux = koff*vec_rn.copies_vec[edge[1]] - temp_kon*(vec_rn.copies_vec[edge[0]])*(vec_rn.copies_vec[node_map[react]])
            #edge_flux = koff*vec_rn.copies_vec[edge[1]] 

            print("Reaction: ", gtostr(rn.network.nodes[edge[0]]['struct']), "+",react," -> ",prod)
            print("Net flux: ",edge_flux)
            print("kon : ",temp_kon)
            print("koff: ",koff)
            print("Reaction data OUTWARD: ")
            print(data)

            total_flux_outedges+=edge_flux
    
    #Now go over all the in edges
    edges_in = rn.network.in_edges(n)
    react_list = []
    if len(edges_in) > 0:
        for edge in edges_in:
            if edge[0] in react_list:
                continue
            data = rn.network.get_edge_data(edge[0],edge[1])
            uid = data['uid']


            #Get generation rates; which would be kon
            temp_kon = vec_rn.kon[uid]

            #Get consumption rates; which is k_off
            std_c = Tensor([1.])
            l_kon = torch.log(temp_kon)
            l_koff = (vec_rn.rxn_score_vec[uid] * 1. / (vec_rn._R * vec_rn._T)) + l_kon + torch.log(std_c)
            koff = torch.exp(l_koff)

            #Get conc. of reactants and products
            prod = gtostr(rn.network.nodes[edge[1]]['struct'])
            #Get other reactant
            react = "".join(sorted(list(set(prod) - set(gtostr(rn.network.nodes[edge[0]]['struct']) ))))
            react_list.append(node_map[react])
            #Net flux from this edge = Generation - consumption
            edge_flux_in = temp_kon*(vec_rn.copies_vec[edge[0]])*(vec_rn.copies_vec[node_map[react]])- koff*vec_rn.copies_vec[edge[1]]
            #edge_flux_in = koff*vec_rn.copies_vec[edge[1]]
            


            print("Reaction: ", prod ," -> ",gtostr(rn.network.nodes[edge[0]]['struct']), "+",react)
            print("Net flux: ",edge_flux_in)
            print("kon : ",temp_kon)
            print("koff: ",koff)
            print("Raction data INWARD: ")
            print(data)

            total_flux_inedges+=edge_flux_in
    net_node_flux = total_flux_outedges + total_flux_inedges
    
    return(net_node_flux)
    
pathway = []
kon_sumarray = []
total_con_rate = {}
net_flux = {}
for n in rn.network.nodes():
    
    n_str = gtostr(rn.network.nodes[n]['struct']) 
    
    paths = [n_str]
    kon_sum = 0
    temp_node = n
    max_edge = True
    consumption_rate = 0
    if n < len(rn.network.nodes()):#num_monomers:
#         print("Current node: ")
#         print(n_str)
        while max_edge:
            max_edge = get_max_edge(temp_node)
            if max_edge:
                total_con_rate[gtostr(rn.network.nodes[temp_node]['struct'])] = max_edge[2]
                
                temp_node = max_edge[1]
                kon_sum += max_edge[0].item()
                
                
#                 print("Next node: ")
#                 print(temp_node)

                paths.append(gtostr(rn.network.nodes[temp_node]['struct']))
            else:
                break
        pathway.append(paths)
        kon_sumarray.append(kon_sum)
        paths=[]
    print("-------------------------------------------------------------------------------")
    print("-------------------------------------------------------------------------------")
    print("|                                                                             |")
    node_flux = get_node_flux(n)
    net_flux[gtostr(rn.network.nodes[n]['struct'])] = node_flux
    print("|                                                                             |")
    print("-------------------------------------------------------------------------------")
    print("-------------------------------------------------------------------------------")

print(pathway)
print(kon_sumarray)

#print(total_con_rate)

In [ ]:
for k,v in sorted(net_flux.items(),key=lambda x : x[1]):
    print(k," : ", v)

print(vec_rn.copies_vec)
print(vec_rn.kon)

In [ ]:
print(solution)
poly_system = EquilibriumSolver(rn)
solution = poly_system.solve(init_val=vec_rn.copies_vec.detach().numpy().tolist())
#solution = poly_system.solve(verifyBool = False)
if solution == None:
    print("No Equilibrium solution")
else:
    print(solution)
    print("Equilibrium expected yield: ", 100 * solution[-1] / min(vec_rn.initial_copies[:vec_rn.num_monomers]), '%')
print(vec_rn.kon)

In [ ]:
print(solution)
poly_system = EquilibriumSolver(rn)
solution = poly_system.solve(verifyBool = False)
if solution == None:
    print("No Equilibrium solution")
else:
    print(solution)
    print("Equilibrium expected yield: ", 100 * solution[-1] / min(vec_rn.initial_copies[:vec_rn.num_monomers]), '%')
print(vec_rn.kon)

In [ ]:
def corr_matrix(M):
    return(np.corrcoef(M))

#Select data range to calculate coorelations
yield_thresh_min = 0.8
yield_thresh_max = 0.9
mask1 = (np.array(optim.final_yields) >= yield_thresh_min) & (np.array(optim.final_yields) < yield_thresh_max)
print(mask1)

coor_params = np.zeros((6,1))
for i in range(len(mask1)):
    if mask1[i]:
        coor_params = np.concatenate((coor_params,np.reshape(optim.final_solns[i].numpy(),(6,1))),axis=1)

corr_matrix = corr_matrix(coor_params)

# %matplotlib notebook
fig_c,ax_c = plt.subplots()
hm = ax_c.imshow(corr_matrix)
ax_labels=['k1','k2','k3','k4','k5','k6']
ax_c.set_xticks(np.arange(len(ax_labels)))
ax_c.set_yticks(np.arange(len(ax_labels)))
ax_c.set_xticklabels(ax_labels)
ax_c.set_yticklabels(ax_labels)
plt.setp(ax_c.get_xticklabels(),rotation=45, ha='center',fontsize=10,va='top')
plt.setp(ax_c.get_yticklabels(),va='center',fontsize=10)
fig_c.colorbar(hm,ax=ax_c,aspect=40,label="Corr Coeff",orientation='horizontal',panchor=(0.5,0.0),pad=0.2,shrink=0.5)
# ax_c.grid()

plt.show()
label = "Number of samples: %.2f %s" %(coor_params.shape[1]*100/len(optim.final_solns),"%")
yield_label = "%.1f - %.1f Yield range\n" %(yield_thresh_min,yield_thresh_max)
ax_c.set_title(yield_label+label)

Clearly, the equilibrium reached by the system still matches the equilibrium solution. We have however found a set of parameters that can increase available complete AP2 at some point before equilibrium to levels significantly higher than at equilibrium. We don't observe any trapping, but have uncovered an interesting effect. 

Now we'll move on to looking at ARP23. This is 7 subunits, which drastically increases the number of possible reactions. Expect longer runtimes. 